In [1]:
%cd ..\packages\bracketeer

H:\pybin\packages\bracketeer


In [38]:
from bracket_builder import bracketeer
# directory structure should be /.../datapath/files.csv
datapath = r'data/'
submissionPath = 'submit.csv'
outputFilePath = 'output.png'
b = bracketeer(2017, datapath)
b.make_bracket(submissionPath, outputFilePath)

In [56]:
import binarytree as bt
from binarytree import Node
import os
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [4]:
bracket = bt.build(range(24))

In [5]:
print bracket


                    ___________________0___________
                   /                               \
          ________1________                      ___2___
         /                 \                    /       \
     ___3___             ___4___              _5        _6
    /       \           /       \            /  \      /  \
  _7        _8        _9        _10        _11   12   13   14
 /  \      /  \      /  \      /   \      /
15   16   17   18   19   20   21    22   23



In [6]:
teams_df = pd.read_csv(os.path.join(datapath, 'Teams.csv'))
seeds_df = pd.read_csv(os.path.join(datapath, 'TourneySeeds.csv'))


In [7]:
teams_df.head()

,Team_Id,Team_Name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [8]:
seeds_df.head()

,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [10]:
df = seeds_df.merge(teams_df, left_on='Team', right_on='Team_Id')
df = df.loc[df['Season'] == 2017, ['Season', 'Seed', 'Team_Id', 'Team_Name']].reset_index(drop=True)

In [11]:
# create a binary tree and pipe things in the right place - create with seeds


df[df['Team_Name'].isin(['Gonzaga', 'South Carolina', 'Oregon', 'North Carolina'])]

,Season,Seed,Team_Id,Team_Name
14,2017,Z01,1314,North Carolina
38,2017,W07,1376,South Carolina
50,2017,Y03,1332,Oregon
51,2017,X01,1211,Gonzaga


In [12]:
import itertools

seedlist = [a+b for a,b in list(itertools.product(list('WXYZ'), [str(n).zfill(2) for n in range(1,17)]))]

In [14]:
#seedlist
btree = bt.build(range(127))
#+ seedlist)
#print btree
bracket_dict = dict(zip(range(127), range(63) + seedlist))

In [26]:
bkt = bt.build(bracket_dict.keys())

In [43]:
bracket_dict[127] = 'W11a'
bracket_dict[128] = 'W11b'
bracket_dict[129] = 'W16a'
bracket_dict[130] = 'W16b'
bracket_dict[131] = 'Y11a'
bracket_dict[132] = 'Y11b'
bracket_dict[133] = 'Z16a'
bracket_dict[134] = 'Z16b'

bkt[73].left = Node(127)
bkt[73].right = Node(128)
bkt[78].left = Node(129)
bkt[78].right = Node(130)
bkt[105].left = Node(131)
bkt[105].right = Node(132)
bkt[126].left = Node(133)
bkt[126].right = Node(134)



# W11   73
# W16   78 
# Y11   105
# Z16   126


In [50]:
with open("temp.txt", 'w') as f:
    f.write(str(bkt))

In [53]:
from matchup_locs.ml2017 import matchup_locs
print(matchup_locs)

{1: (372, 32), 2: (372, 50), 3: (30, 328), 4: (30, 346), 5: (695, 325), 6: (695, 343), 7: (370, 642), 8: (370, 659), 9: (30, 38), 10: (30, 55), 11: (30, 74), 12: (30, 92), 13: (30, 110), 14: (30, 128), 15: (30, 146), 16: (30, 164), 17: (30, 182), 18: (30, 199), 19: (30, 218), 20: (30, 236), 21: (30, 254), 22: (30, 272), 23: (30, 290), 24: (30, 308), 25: (30, 370), 26: (30, 388), 27: (30, 406), 28: (30, 424), 29: (30, 442), 30: (30, 460), 31: (30, 478), 32: (30, 496), 33: (30, 514), 34: (30, 532), 35: (30, 550), 36: (30, 567), 37: (30, 586), 38: (30, 604), 39: (30, 622), 40: (30, 640), 41: (815, 38), 42: (815, 55), 43: (815, 74), 44: (815, 92), 45: (815, 110), 46: (815, 128), 47: (815, 146), 48: (815, 164), 49: (815, 182), 50: (815, 199), 51: (815, 218), 52: (815, 236), 53: (815, 254), 54: (815, 272), 55: (815, 290), 56: (815, 308), 57: (815, 370), 58: (815, 388), 59: (815, 406), 60: (815, 424), 61: (815, 442), 62: (815, 460), 63: (815, 478), 64: (815, 496), 65: (815, 514), 66: (815, 53

In [64]:
slotdata = zip(matchup_locs.values(), bracket_dict.values())

In [68]:
# Create bracket image
# relevant:
# https://stackoverflow.com/questions/26649716/how-to-show-pil-image-in-ipython-notebook
emptyBracketPath = 'empty_brackets/2017.jpg'
img = Image.open(emptyBracketPath)
draw = ImageDraw.Draw(img)
# font = ImageFont.truetype(<font-file>, <font-size>)
# draw.text((x, y),"Sample Text",(r,g,b))
for slot0, slot1 in slotdata:
    draw.text(slot0, slot1, (0, 0, 0))

dpi = 72
margin = 0.05  # (5% of the width/height of the figure...)
xpixels, ypixels = 940, 700

# Make a figure big enough to accomodate an axis of xpixels by ypixels
# as well as the ticklabels, etc...
figsize = (1 + margin) * ypixels / dpi, (1 + margin) * xpixels / dpi
fig = plt.figure(figsize=figsize, dpi=dpi)
# Make the axis the right size...
ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

ax.imshow(np.asarray(img))
# plt.show() # for in notebook
img.save(outputFilePath)

TypeError: argument of type 'int' is not iterable

In [69]:
slotdata

[((372, 32), 0),
 ((372, 50), 1),
 ((30, 328), 2),
 ((30, 346), 3),
 ((695, 325), 4),
 ((695, 343), 5),
 ((370, 642), 6),
 ((370, 659), 7),
 ((30, 38), 8),
 ((30, 55), 9),
 ((30, 74), 10),
 ((30, 92), 11),
 ((30, 110), 12),
 ((30, 128), 13),
 ((30, 146), 14),
 ((30, 164), 15),
 ((30, 182), 16),
 ((30, 199), 17),
 ((30, 218), 18),
 ((30, 236), 19),
 ((30, 254), 20),
 ((30, 272), 21),
 ((30, 290), 22),
 ((30, 308), 23),
 ((30, 370), 24),
 ((30, 388), 25),
 ((30, 406), 26),
 ((30, 424), 27),
 ((30, 442), 28),
 ((30, 460), 29),
 ((30, 478), 30),
 ((30, 496), 31),
 ((30, 514), 32),
 ((30, 532), 33),
 ((30, 550), 34),
 ((30, 567), 35),
 ((30, 586), 36),
 ((30, 604), 37),
 ((30, 622), 38),
 ((30, 640), 39),
 ((815, 38), 40),
 ((815, 55), 41),
 ((815, 74), 42),
 ((815, 92), 43),
 ((815, 110), 44),
 ((815, 128), 45),
 ((815, 146), 46),
 ((815, 164), 47),
 ((815, 182), 48),
 ((815, 199), 49),
 ((815, 218), 50),
 ((815, 236), 51),
 ((815, 254), 52),
 ((815, 272), 53),
 ((815, 290), 54),
 ((815, 3

In [ ]:
need to fix slotdata, right now looks like a list of tuples of tuples?